In [2]:
import polars as pl

In [3]:
games_df = pl.read_csv("games_df.csv")
games_df = games_df.filter(
    (pl.col("forbidMoving") == True)
    & (pl.col("forbidZooming") == False)
    & (pl.col("forbidRotating") == False)
    & (pl.col("roundCount") == 5)
    & (pl.col("roundTime") == 40)
    & (pl.col("mapSlug") == "world")
)
games_df.head(2)

game_id,forbidMoving,forbidZooming,forbidRotating,roundCount,mapSlug,roundTime,isSinglePlayer,scoringType,context,date
str,bool,bool,bool,i64,str,i64,bool,str,str,str
"""43b82fcf-29d8-4daf-84e9-80e029…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:06:54.013Z"""
"""14cf63a7-1289-4b09-9ab8-75eb4f…",true,false,false,5,"""world""",40,false,"""Unknown""","""""","""2024-10-30T18:11:55.588Z"""


In [4]:
guesses_df = pl.read_csv("guesses_df.csv")
guesses_df = guesses_df.filter(
    pl.col("gameId").is_in(games_df["game_id"].unique())
)
guesses_df.head(2)

roundNumber,lat,lng,distance,time,score,wasCorrect,gameId,playerName,code,playerId,payload,timestamp,lobby,countryGuess,coordinateGuess,battleRoyaleGameState,battleRoyalePlayer,duel,bullseye,player,country_code
i64,f64,f64,f64,i64,i64,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
3,51.47018,5.786652,117653.422716,40,4621,false,"""ea8d34ee-d2c9-4283-834e-f0438f…","""moxxiq""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-08-21T18:47:34.5599291Z""",null,null,null,null,null,null,null,"""moxxiq""","""NL"""
3,-34.665525,-58.416663,3.9950e6,34,343,false,"""2d460ac2-f967-420b-9867-cefe08…","""Ishenko01y""","""LiveChallengeLeaderboardUpdate""",null,null,"""2024-12-11T18:39:45.8073149Z""",null,null,null,null,null,null,null,"""Ishenko01y""","""AR"""


In [5]:
df = guesses_df.group_by("country_code").agg(
    total_rounds=pl.col("country_code").count(),
    avg_score=pl.col("score").mean().round(2),
    median_score=pl.col("score").median().cast(pl.Int32),
)
df.sample(5)

country_code,total_rounds,avg_score,median_score
str,u32,f64,i32
"""IN""",42,2430.74,2445
"""IE""",42,3861.26,4478
"""PS""",3,4197.67,4971
"""CL""",23,1279.87,893
"""BT""",5,2980.0,4524


In [6]:
df.filter(pl.col("total_rounds") > 20).sort(by="avg_score", descending=True)[:5]

country_code,total_rounds,avg_score,median_score
str,u32,f64,i32
"""NL""",57,4291.26,4621
"""KR""",29,4195.72,4530
"""CH""",25,4104.32,4720
"""AT""",74,3886.22,4256
"""IE""",42,3861.26,4478


In [8]:
df.filter(pl.col("total_rounds") > 20).sort(by="avg_score", descending=False)[:10]

country_code,total_rounds,avg_score,median_score
str,u32,f64,i32
"""BR""",38,686.05,475
"""EC""",33,1198.58,888
"""CL""",23,1279.87,893
"""KE""",25,1291.64,380
"""BO""",26,1300.15,925
"""RU""",30,1362.57,1107
"""MX""",75,1415.64,400
"""CO""",57,1447.12,637
"""US""",113,1489.63,1052
